In [1]:
import os
import pandas as pd
from decimal import Decimal
import numpy as np
from datetime import datetime, timedelta
from dotenv import load_dotenv
import yfinance as yf


import matplotlib.pyplot as plt
import plotly.express as px
import pytz  # Make sure to import pytz for timezone handling
import seaborn as sns


import requests
import csv
import json

import warnings



from sklearn.linear_model import LinearRegression

In [2]:
load_dotenv()

API_KEY = os.getenv("alpha_vantage_api_key")

In [3]:
# Addtional setting session
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
pd.options.mode.copy_on_write = True

# PE TTM valuation

In [21]:
# Parameters section

alpha_vantage_api_key = API_KEY # FREE TIER API rate limit is 25 requests per day
alpha_vantage_function = {
    'core':[
        'TIME_SERIES_INTRADA'
        ,'TIME_SERIES_DAILY' # this is daily time series quote
        ,'TIME_SERIES_DAILY_ADJUSTED' # this is daily time series adjusted by split/dividend-adjusted
        ,'GLOBAL_QUOTE'
    ]
    ,'fundmental':[
    'INCOME_STATEMENT'
    ,'BALANCE_SHEET' # this is daily time series quote
    ,'CASH_FLOW' # this is daily time series adjusted by split/dividend-adjusted
    ,'EARNINGS'
    ,'EARNINGS_CALENDAR'
]
}

# # Custmized
ticker_symbols = [

    # 'GOOG'
    'LPG'
    ,'STNG'
    ,'GLNG'
    # ,'COST'
    # ,'MSFT'
    # ,'TSM'
]

# ticker_symbols = [
# 'AAP',
#  'ADT',
#  'AN',
#  'ARMK',
#  'AS',
#  'BBWI',
#  'BC',
#  'BFAM',
#  'BIRK',
#  'BLD',
#  'BROS',
#  'BURL',
#  'BYD',
#  'CAVA',
#  'CHDN',
#  'CHH',
#  'COLM',
#  'CPNG',
#  'CPRI',
#  'CRI',
#  'CROX',
#  'CVNA',
#  'DASH',
#  'DDS',
#  'DKNG',
#  'DKS',
#  'DUOL',
#  'ETSY',
#  'FIVE',
#  'FND',
#  'GAP',
#  'GME',
#  'GNTX',
#  'H',
#  'HOG',
#  'HRB',
#  'JWN',
#  'KSS',
#  'LAD',
#  'LCID',
#  'LEA',
#  'LEG',
# #  'LEN.B',
#  'LNW',
#  'LOPE',
#  'M',
#  'MAT',
#  'MTN',
#  'MUSA',
#  'NWL',
#  'OLLI',
#  'PAG',
#  'PENN',
#  'PII',
#  'PLNT',
#  'PVH',
#  'QS',
#  'RH',
#  'RIVN',
#  'SCI',
#  'SKX',
#  'SN',
#  'THO',
#  'TNL',
#  'TOL',
#  'TPX',
#  'TXRH',
#  'UA',
#  'UAA',
#  'VAC',
#  'VFC',
#  'VSTS',
#  'W',
#  'WEN',
#  'WH',
#  'WHR',
#  'WING',
#  'WSM',
#  'YETI'
# ]


# Time intelligent parameters
window_days = 90
end_date = datetime.now()
start_date = end_date - timedelta(days=window_days)
earning_calendar = [
    3  # this will return next 1 qtr forecast earning; nowadays the earning calendar only shows the next 1 qtr forecast earning
    ,6  # this will return next 2 qtr forecast earning
    ,12  # this will return next 4 qtr forecast earning
]

PE_yr_range = 4 # this will return x-1 yr PE range, cuz you cannot get current year end PE

ticker_dict_json = {}
ticker_dict_pd = {}

In [22]:
def convert_to_numeric_or_zero(value):
    try:
        # Try to convert the string to a float (or int)
        num = round(float(value), 2)
        return num  # Return the original value if it can be converted to a number
    except ValueError:
        # If conversion fails, return '0'
        return 0

In [23]:
# Daily quote section
for j, symbol in enumerate(ticker_symbols):

    print(j, symbol)
    # STOCK SPLIT FACTOR section
    url = f'https://www.alphavantage.co/query?function=SPLITS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            if len(value) > 0:
                stock_split_record_df = pd.DataFrame(value)
                stock_split_record_df['split_factor'] = pd.to_numeric(stock_split_record_df['split_factor'], errors='coerce') # change split_factor series to numeric data
                stock_split_record_df['effective_date'] = pd.to_datetime(stock_split_record_df['effective_date'])
            else:
                stock_split_record_df = pd.DataFrame()
                stock_split_record_df['split_factor'] = 1
                stock_split_record_df['effective_date'] = datetime.today()


    # Daily quote section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}&outputsize=full'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Time Series (Daily)':


            selected_cols = [
                '4. close'
            ]

            Daily_stock_df = pd.DataFrame(value).transpose()[selected_cols] # tranpose the dataframe and sub select selected cols

            # Rename columns
            Daily_stock_df.rename(
                columns={
                    '4. close': 'stock_price'
                    }
                ,inplace=True
                )
            
            Daily_stock_df["stock_price"] = Daily_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
            Daily_stock_df["stock_price"] = Daily_stock_df["stock_price"].round(2)
            Daily_stock_df.index = pd.to_datetime(Daily_stock_df.index)


    for date_i in Daily_stock_df.index.date:
        for date_j in stock_split_record_df['effective_date'].dt.date:
            if date_i == date_j:

                # stock price to divided the split factor
                Daily_stock_df.loc[Daily_stock_df.index.date < date_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.date == date_j].values[0])


    # MA200 calculation
    Daily_stock_df['MA200'] = Daily_stock_df.sort_index(ascending=True)['stock_price'].rolling(window=200).mean()

    # Calculate the slope of the MA200 using linear regression
    def calculate_slope(series):
        # Create a time index (0, 1, 2, ..., n) for the linear regression
        x = np.arange(len(series)).reshape(-1, 1)
        y = series.values.reshape(-1, 1)

        # Fit the linear regression model
        model = LinearRegression()
        model.fit(x, y)

        # Return the slope
        return model.coef_[0][0]


    # Apply the slope calculation to the MA200 values
    Daily_stock_df['MA200_slope'] = Daily_stock_df.sort_index(ascending=True)['MA200'].rolling(window=20).apply(calculate_slope, raw=False)



    # Monthly quote section
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Monthly Time Series':
            Monthly_stock_df = pd.DataFrame(value)

    Monthly_stock_df = Monthly_stock_df.transpose()
    Monthly_stock_df.index = pd.to_datetime(Monthly_stock_df.index)


    filter_1 = (Monthly_stock_df.index.year.isin(range((datetime.today().year - PE_yr_range) ,datetime.today().year)))
    filter_2 = (Monthly_stock_df.index.month == 12) # month = 12 to get the year end closing price

    selected_cols = [
        '4. close'
    ]

    Monthly_stock_df = Monthly_stock_df[
        filter_1
        & filter_2
    ][selected_cols]

    # Rename columns
    Monthly_stock_df.rename(
        columns={
            '4. close': 'stock_price'
            }
        ,inplace=True
        )

    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].round(2)

    # modify stock price based on stock split
    for year_i in Monthly_stock_df.index.year:
        for year_j in stock_split_record_df['effective_date'].dt.year:
            if year_i == year_j:

                # stock price to divided the split factor
                Monthly_stock_df.loc[Monthly_stock_df.index.year < year_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.year == year_j].values[0])



    # Earning section
    # past earnings from alpha vintage API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            annualEPS_df = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


            annualEPS_df['fiscalDateEnding'] = pd.to_datetime(annualEPS_df['fiscalDateEnding']).dt.year

            annualEPS_df = annualEPS_df[
                annualEPS_df['fiscalDateEnding'].isin(
                    range(
                        (datetime.today().year - PE_yr_range) 
                        ,datetime.today().year
                            )
                            )
                            ]

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    annualEPS_df[f'{col}'] = annualEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue
            
            # clean annualEPS_df
            annualEPS_df = annualEPS_df.sort_values('reportedEPS', ascending=False).drop_duplicates('fiscalDateEnding')
            annualEPS_df = annualEPS_df.sort_values('fiscalDateEnding', ascending=False).reset_index(drop=True)

            # calculate metrics
            if Monthly_stock_df.shape[0] <= annualEPS_df.shape[0]: # if Monthly_stock_df has less records than annualEPS_df, choose the mini length record
                annualEPS_df = annualEPS_df[:(Monthly_stock_df.shape[0])]

                annualEPS_df["PE"] = Monthly_stock_df["stock_price"].values / annualEPS_df['reportedEPS'].values
                annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"] = annualEPS_df["PE"].mean().round(2)
                annualEPS_df[f"PE_{PE_yr_range-1}yr_std"] = np.std(annualEPS_df["PE"]).round(2)
                annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_+"] = (annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"] + annualEPS_df[f"PE_{PE_yr_range-1}yr_std"]).round(2) # 这个是PE的波动范围上限
                annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_-"] = (annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"] - annualEPS_df[f"PE_{PE_yr_range-1}yr_std"]).round(2) # 这个是PE的波动范围下限



        if key == 'quarterlyEarnings':

            selected_cols = [
                'reportedDate'
                ,'reportedEPS'
            ]

            qtrEPS_df = pd.DataFrame(value)[selected_cols] # tranpose the dataframe and sub select selected cols
            qtrEPS_df['reportedDate'] = pd.to_datetime(qtrEPS_df['reportedDate'])

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    qtrEPS_df[col] = qtrEPS_df[col].astype(str).apply(lambda x: float(x) if x not in [None, 'None', 'nan', 'NaN'] else float(0))
                else:
                    continue


    # # forecast 1 qtr earnings from alpha vantage API
    # for i in earning_calendar: comment out the for loop in case of future usage, i can be the parameter of {}month
    CSV_URL = f'https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&symbol={symbol}&horizon=12month&apikey={alpha_vantage_api_key}'
    with requests.Session() as s:
        download = s.get(CSV_URL)
        decoded_content = download.content.decode('utf-8')
        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        my_list = list(cr)

        forecast_earanings_df = pd.DataFrame(
            columns=my_list[0]
            ,data=my_list[1::]
            )
        
        if forecast_earanings_df['estimate'].head(1).values != '':
            latest_projected_EPS = float(forecast_earanings_df['estimate'].head(1).values)
        else:
            latest_projected_EPS = 0


    # forecast 1 year earnings from yf API, forwardPE, PEG
    # yf data
    yf_data = yf.Ticker(symbol).info
    # alpha vintage data
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()
    
    for key, value in data.items():
        if key == 'PEGRatio':
            PEG_12month_projected = convert_to_numeric_or_zero(value=value)


        if key == 'ForwardPE':
           PE_12month_projected = convert_to_numeric_or_zero(value=value)


    forecast_earnings_keys = [
        'forwardEps'
    ]


    if not all(key in yf_data.keys() for key in forecast_earnings_keys):
        # Handle the case where one or more keys are missing
        EPS_12month_projected = 0
    else:
        EPS_12month_projected = yf_data['forwardEps'] # 代表了截止下一个日历年结束的EPS, next year forecasted EPS


    # US Treasury section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=daily&maturity=10year&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            US_T_10yrs_df = pd.DataFrame(value)
            US_T_10yrs_df['value'] = pd.to_numeric(US_T_10yrs_df['value'], errors='coerce') # change dataframe value to numeric data
            US_T_10yrs_YTM = US_T_10yrs_df['value'][0]



    # COMPANY OVERVIEW
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    
    for key, value in data.items():
        if key  == 'MarketCapitalization':
            stock_mkt_cap = float(value)



    # INCOME STATEMENT
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()


    for key, value in data.items():
        if key == 'annualReports':
            annual_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_income_df = annual_income_df.sort_values('fiscalDateEnding', ascending=True)

            # annual measurements cols
            annual_income_metric_cols = annual_income_df.columns[1:]

            # Calculate quarter-over-quarter change percentage
            for column in annual_income_metric_cols:  # Exclude the 'fiscalDateEnding' column
                annual_income_df[column] = pd.to_numeric(annual_income_df[column], errors='coerce')
                annual_income_df[f'{column}_YoY'] = annual_income_df[column].pct_change() * 100 
            
            annual_income_YoY_metric_cols = [col for col in annual_income_df.columns if 'YoY' in col]

            # ratios of income statement calculation
            annual_income_df['gross_margin_%'] = (annual_income_df['grossProfit'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['operating_margin_%'] = (annual_income_df['operatingIncome'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['net_margin_%'] = (annual_income_df['netIncome'] / annual_income_df['totalRevenue']) * 100



        if key == 'quarterlyReports':
            qtr_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_income_df = qtr_income_df.sort_values('fiscalDateEnding', ascending=True)

            # qtr measurements cols
            qtr_income_metric_cols = qtr_income_df.columns[1:]
            
            # Calculate quarter-over-quarter change percentage
            for column in qtr_income_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_income_df[column] = pd.to_numeric(qtr_income_df[column], errors='coerce')
                qtr_income_df[f'{column}_QoQ'] = qtr_income_df[column].pct_change() * 100

            qtr_income_QoQ_metric_cols = [col for col in qtr_income_df.columns if 'QoQ' in col]

            # ratios of income statement calculation
            qtr_income_df['gross_margin_%'] = (qtr_income_df['grossProfit'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['operating_margin_%'] = (qtr_income_df['operatingIncome'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['net_margin_%'] = (qtr_income_df['netIncome'] / qtr_income_df['totalRevenue']) * 100

    income_ratio_cols = [
        'gross_margin_%'
        ,'operating_margin_%'
        ,'net_margin_%'
        ,'netIncome'
        ]
    
    

    # BALANCESHEET
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():

        if key == 'annualReports':
            annual_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_balancesheet_df = annual_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in annual_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                annual_balancesheet_df[column] = pd.to_numeric(annual_balancesheet_df[column], errors='coerce')

            annual_balancesheet_df['current_ratio'] = (annual_balancesheet_df['totalCurrentAssets'] / annual_balancesheet_df['totalCurrentLiabilities'])
            annual_balancesheet_df['working_capital'] = annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['totalCurrentLiabilities']
            annual_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (annual_balancesheet_df['longTermDebt'] / annual_balancesheet_df['working_capital'])
            annual_balancesheet_df['debtEquity_ratio'] = (annual_balancesheet_df['totalLiabilities'] / annual_balancesheet_df['totalShareholderEquity'])
            annual_balancesheet_df['quick_ratio'] = ((annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['inventory']) / annual_balancesheet_df['totalCurrentLiabilities'])

            
        if key == 'quarterlyReports':
            qtr_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_balancesheet_df = qtr_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in qtr_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_balancesheet_df[column] = pd.to_numeric(qtr_balancesheet_df[column], errors='coerce')

    
            qtr_balancesheet_df['current_ratio'] = (qtr_balancesheet_df['totalCurrentAssets'] / qtr_balancesheet_df['totalCurrentLiabilities'])
            qtr_balancesheet_df['working_capital'] = qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['totalCurrentLiabilities']
            qtr_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (qtr_balancesheet_df['longTermDebt'] / qtr_balancesheet_df['working_capital'])
            qtr_balancesheet_df['debtEquity_ratio'] = (qtr_balancesheet_df['totalLiabilities'] / qtr_balancesheet_df['totalShareholderEquity'])
            qtr_balancesheet_df['quick_ratio'] = ((qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['inventory']) / qtr_balancesheet_df['totalCurrentLiabilities'])

            qtr_balancesheet_df['BVPS_latest'] = round(
                qtr_balancesheet_df.tail(1)['totalShareholderEquity'].sum() / qtr_balancesheet_df.tail(1)['commonStockSharesOutstanding']
                ,2
                )



    balancesheet_ratio_cols = [
        'current_ratio'
        ,'working_capital'
        ,'longTermDebt_to_workingCp_ratio'
        ,'debtEquity_ratio'
        ,'quick_ratio'
        ,'totalShareholderEquity'
    ]


    # CASHFLOW STATEMENT
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():

        if key == 'annualReports':
            annual_cashflow_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_cashflow_df = annual_cashflow_df.sort_values('fiscalDateEnding', ascending=True)

            for column in annual_cashflow_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                annual_cashflow_df[column] = pd.to_numeric(annual_cashflow_df[column], errors='coerce')
                annual_cashflow_df[f'{column}_YoY'] = annual_cashflow_df[column].pct_change() * 100

            
        if key == 'quarterlyReports':
            qtr_cashflow_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_cashflow_df = qtr_cashflow_df.sort_values('fiscalDateEnding', ascending=True)

            for column in qtr_cashflow_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_cashflow_df[column] = pd.to_numeric(qtr_cashflow_df[column], errors='coerce')
                qtr_cashflow_df[f'{column}_YoY'] = qtr_cashflow_df[column].pct_change() * 100


    stock_ratios_annual_consolidate_df = pd.DataFrame()
    stock_ratios_qtr_consolidate_df = pd.DataFrame()

    stock_ratios_annual_consolidate_df[income_ratio_cols] = annual_income_df[income_ratio_cols]
    stock_ratios_annual_consolidate_df[balancesheet_ratio_cols] = annual_balancesheet_df[balancesheet_ratio_cols]

    stock_ratios_qtr_consolidate_df[income_ratio_cols] = qtr_income_df[income_ratio_cols]
    stock_ratios_qtr_consolidate_df[balancesheet_ratio_cols] = qtr_balancesheet_df[balancesheet_ratio_cols]

    # calculating new consolidated metrics 
    # annual df
    stock_ratios_annual_consolidate_df['ROE_%'] = (annual_income_df['netIncome'] / annual_balancesheet_df['totalShareholderEquity']) * 100
    stock_ratios_annual_consolidate_df['liquidation_value'] = ((annual_balancesheet_df['totalAssets'] - annual_balancesheet_df['intangibleAssets']) - annual_balancesheet_df['totalLiabilities'])
    stock_ratios_annual_consolidate_df['liquidation_mktcap_ratio_%'] = round((stock_ratios_annual_consolidate_df['liquidation_value'] / stock_mkt_cap) * 100, 2)
    stock_ratios_annual_consolidate_df['fiscalDateEnding'] = annual_balancesheet_df['fiscalDateEnding']

    # qtr df
    stock_ratios_qtr_consolidate_df['ROE_%'] = (qtr_income_df['netIncome'] / qtr_balancesheet_df['totalShareholderEquity']) * 100
    stock_ratios_qtr_consolidate_df['liquidation_value'] = ((qtr_balancesheet_df['totalAssets'] - qtr_balancesheet_df['intangibleAssets']) - qtr_balancesheet_df['totalLiabilities'])
    stock_ratios_qtr_consolidate_df['liquidation_mktcap_ratio_%'] = round((stock_ratios_qtr_consolidate_df['liquidation_value'] / stock_mkt_cap) * 100, 2)
    stock_ratios_qtr_consolidate_df['fiscalDateEnding'] = qtr_balancesheet_df['fiscalDateEnding']





    # Consolidated section
    stock_consolidate_df = Daily_stock_df.head(window_days)


    stock_consolidate_df_date = stock_consolidate_df.index
    for i in stock_consolidate_df_date:
                
        # Filter the DataFrame to include only dates(index) less than or equal to the target date
        filtered_qtrEPS_df = qtrEPS_df[qtrEPS_df['reportedDate'] < i]

        # Select the first four rows from the past_qtrs_EPS
        past_4_qtrs_EPS = filtered_qtrEPS_df.head(4) 
        past_3_qtrs_EPS = filtered_qtrEPS_df.head(3)
        past_1_qtr_EPS = filtered_qtrEPS_df.head(1)

        # Calculate the sum of the numeric values in the selected rows
        EPS_TTM = past_4_qtrs_EPS['reportedEPS'].values.sum()
        EPS_curr_qtr = past_1_qtr_EPS['reportedEPS'].values.sum()

        # assign each index row with the EPS_TTM
        stock_consolidate_df.loc[i, "EPS_TTM"] = EPS_TTM
        stock_consolidate_df.loc[i, "EPS_currentQtr"] = EPS_curr_qtr

        if i == max(stock_consolidate_df.index):
            EPS_nextQtr_projected = latest_projected_EPS + past_3_qtrs_EPS['reportedEPS'].values.sum()  # This metrics is the past 3 qtrs post EPS + 1 projected EPS
        else:
            continue

        stock_consolidate_df["EPS_nextQtr"] = latest_projected_EPS
        stock_consolidate_df["EPS_nextQtr_TTM"] = EPS_nextQtr_projected


    

    # stock's stats
    stock_consolidate_df["Ticker"] = symbol
    stock_consolidate_df["PE_1yr_forward"] = round(PE_12month_projected, 2)
    stock_consolidate_df["PE_TTM"] = (stock_consolidate_df["stock_price"] / stock_consolidate_df["EPS_TTM"]).round(2)
    stock_consolidate_df["PE_TTM_avg"] = stock_consolidate_df["PE_TTM"].mean().round(2)
    stock_consolidate_df["PE_TTM_std"] = np.std(stock_consolidate_df["PE_TTM"]).round(2)
    stock_consolidate_df["PE_TTM_volatility_+"] = (stock_consolidate_df["PE_TTM_avg"] + stock_consolidate_df["PE_TTM_std"]).round(2) # 这个是PE的波动范围上限
    stock_consolidate_df["PE_TTM_volatility_-"] = (stock_consolidate_df["PE_TTM_avg"] - stock_consolidate_df["PE_TTM_std"]).round(2) # 这个是PE的波动范围下限

    stock_consolidate_df["relative_valuation_TTM_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * stock_consolidate_df["EPS_TTM"]).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_TTM_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * stock_consolidate_df["EPS_TTM"]).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_TTM_median"] = (np.median([stock_consolidate_df["relative_valuation_TTM_+"][0], stock_consolidate_df["relative_valuation_TTM_-"][0]])).round(2) #这个是根据最新TTM PE估值的价格中位数

    stock_consolidate_df["relative_valuation_nextQuater_projected_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * stock_consolidate_df["EPS_nextQtr_TTM"]).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_nextQuater_projected_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * stock_consolidate_df["EPS_nextQtr_TTM"]).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_nextQuater_projected_median"] = (np.median([stock_consolidate_df["relative_valuation_nextQuater_projected_+"][0], stock_consolidate_df["relative_valuation_nextQuater_projected_-"][0]])).round(2) #这个是根据3 qtrs post EPS + 1 projected EPS 得出PE估值的价格中位数

    stock_consolidate_df[f"{window_days}_price_min"] = stock_consolidate_df["stock_price"].min().round(2)
    stock_consolidate_df[f"{window_days}_price_max"] = stock_consolidate_df["stock_price"].max().round(2)
    stock_consolidate_df[f"{window_days}_price_avg"] = stock_consolidate_df["stock_price"].mean().round(2)
    stock_consolidate_df[f"{window_days}_price_std"] = np.std(stock_consolidate_df["stock_price"]).round(2)

    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_avg"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"].values[0]
    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_std"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_std"].values[0]
    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_volatility_+"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_+"].values[0]
    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_volatility_-"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_-"].values[0]


    stock_consolidate_df["relative_valuation_nextYear_projected_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * EPS_12month_projected).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_nextYear_projected_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * EPS_12month_projected).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_nextYear_projected_median"] = (np.median([stock_consolidate_df["relative_valuation_nextYear_projected_+"][0], stock_consolidate_df["relative_valuation_nextYear_projected_-"][0]])).round(2) #这个是根据next year projected EPS 得出PE估值的价格中位数

    stock_consolidate_df["PEG_next12months"] = PEG_12month_projected
    stock_consolidate_df["EPS_next12months"] = EPS_12month_projected
    stock_consolidate_df["PEG_TTM"] = (stock_consolidate_df["PE_TTM"] / (((EPS_12month_projected - stock_consolidate_df["EPS_TTM"]) / stock_consolidate_df["EPS_TTM"]) * 100)).round(2) # 这个是截止下一年的EPS growth rate所得出的PEG ratio, <1是undervalue的表现
    
    stock_consolidate_df["EPS_nextYr_growthRate"] = (((EPS_12month_projected - stock_consolidate_df["EPS_TTM"]) / stock_consolidate_df["EPS_TTM"]) * 100).round(2)
    stock_consolidate_df["EPS_nextQtr_growthRate"] = (((stock_consolidate_df["EPS_nextQtr_TTM"] - stock_consolidate_df["EPS_TTM"]) / stock_consolidate_df["EPS_TTM"]) * 100).round(2)

    stock_consolidate_df["EarningYield_TTM"] = ((stock_consolidate_df["EPS_TTM"] / stock_consolidate_df["stock_price"]) * 100).round(2)
    stock_consolidate_df["ERP_TTM"] = stock_consolidate_df["EarningYield_TTM"] - US_T_10yrs_YTM # it means a comparison between equity return and 10 years risk free, usually ERP >= 3 for short term invest at least, ERP >= 5 for long term invest 

    stock_consolidate_df['latest_qtr_liquidation_mktcap_ratio_%'] = stock_ratios_qtr_consolidate_df['liquidation_mktcap_ratio_%'].values[-1]
    stock_consolidate_df['BVPS_latest'] = qtr_balancesheet_df['BVPS_latest'].values[-1]

    stock_consolidate_df['MA200_slope'] = Daily_stock_df['MA200_slope']

    stock_consolidate_df['FCF_per_share_TTM'] = round((qtr_cashflow_df['operatingCashflow'] - qtr_cashflow_df['capitalExpenditures']).tail(4).sum() / qtr_balancesheet_df['commonStockSharesOutstanding'].values[-1], 2)
    stock_consolidate_df['PFCF_TTM'] = round(stock_consolidate_df['stock_price'][0]  / stock_consolidate_df['FCF_per_share_TTM'], 2)
    stock_consolidate_df['FCF_yield_TTM'] = round((stock_consolidate_df['FCF_per_share_TTM'] / stock_consolidate_df['stock_price'][0]) * 100, 2) # % return on cash of every single dollar you spent on stock price, the higher the better


    # filter conditions
    conditions = [
    (stock_consolidate_df["stock_price"] < stock_consolidate_df["relative_valuation_TTM_-"]),
    (stock_consolidate_df["stock_price"] > stock_consolidate_df["relative_valuation_TTM_+"]),
    ((stock_consolidate_df["stock_price"] >= stock_consolidate_df["relative_valuation_TTM_-"]) & (stock_consolidate_df["stock_price"] <= stock_consolidate_df["relative_valuation_TTM_+"])),
    ]

    categories = [
        'undervalued'
        ,'overvalued'
        ,'fair'
        ]

    # This KPI assess if the current stock price is under/over/fair to the current relative valuation
    stock_consolidate_df["curr_assessment"] = None

    for condition, category in zip(conditions, categories):
        stock_consolidate_df.loc[condition, "price_valuation_assessment"] = category





    # Append key-value pairs to the dictionary
    selected_cols = [
    "Ticker"
    ,"stock_price"
    ,"EPS_currentQtr"
    ,"EPS_nextQtr"
    ,"EPS_TTM"
    ,"EPS_nextQtr_TTM"
    ,"EPS_next12months"
    ,"PE_1yr_forward"
    ,"PE_TTM"
    ,"PE_TTM_avg"
    ,"PE_TTM_volatility_+"
    ,"PE_TTM_volatility_-"
    ,f"PE_{PE_yr_range-1}yr_avg"
    ,f"PE_{PE_yr_range-1}yr_volatility_+"
    ,f"PE_{PE_yr_range-1}yr_volatility_-"
    ,"relative_valuation_TTM_+"
    ,"relative_valuation_TTM_-"
    ,"relative_valuation_TTM_median"
    ,"relative_valuation_nextQuater_projected_+"
    ,"relative_valuation_nextQuater_projected_-"
    ,"relative_valuation_nextQuater_projected_median"
    ,"relative_valuation_nextYear_projected_+"
    ,"relative_valuation_nextYear_projected_-"
    ,"relative_valuation_nextYear_projected_median"
    ,"price_valuation_assessment"
    ,"EPS_nextQtr_growthRate"
    ,"EPS_nextYr_growthRate"
    ,"PEG_next12months"
    ,"PEG_TTM"
    ,"EarningYield_TTM"
    ,"ERP_TTM"
    ,"latest_qtr_liquidation_mktcap_ratio_%"
    ,"BVPS_latest"
    ,"PFCF_TTM"
    ,"FCF_yield_TTM" 
    ,"MA200_slope"
    ]


    # store each stock info as pd into dictionary
    ticker_dict_pd[symbol] = stock_consolidate_df[selected_cols]
    # transfer pandas dataframe to json format, and each stock info into dictionary
    ticker_dict_json[symbol] = stock_consolidate_df[selected_cols].to_dict()

    if j == 0:
        # screener df creation
        # screener df will store each stock's consolidate df's first row and union them together for screening purposee
        ticker_screen_df = pd.DataFrame(
            columns=selected_cols
        ) 
        stock_consolidate_df_values = stock_consolidate_df[selected_cols].values[0]

        # Insert rows into the DataFrame
        ticker_screen_df.loc[j] = stock_consolidate_df_values

    else:
        stock_consolidate_df_values = stock_consolidate_df[selected_cols].values[0]
        ticker_screen_df.loc[j] = stock_consolidate_df_values


# ticker screen df consolidated metrics
ticker_screen_df['Industry_PE_TTM_avg'] = round(ticker_screen_df['PE_TTM'].mean(), 2)

0 LPG
1 STNG
2 GLNG


In [16]:
ticker_dict_pd['FLNG']

,Ticker,stock_price,EPS_currentQtr,EPS_nextQtr,EPS_TTM,EPS_nextQtr_TTM,EPS_next12months,PE_1yr_forward,PE_TTM,PE_TTM_avg,PE_TTM_volatility_+,PE_TTM_volatility_-,PE_3yr_avg,PE_3yr_volatility_+,PE_3yr_volatility_-,relative_valuation_TTM_+,relative_valuation_TTM_-,relative_valuation_TTM_median,relative_valuation_nextQuater_projected_+,relative_valuation_nextQuater_projected_-,relative_valuation_nextQuater_projected_median,relative_valuation_nextYear_projected_+,relative_valuation_nextYear_projected_-,relative_valuation_nextYear_projected_median,price_valuation_assessment,EPS_nextQtr_growthRate,EPS_nextYr_growthRate,PEG_next12months,PEG_TTM,EarningYield_TTM,ERP_TTM,latest_qtr_liquidation_mktcap_ratio_%,BVPS_latest,PFCF_TTM,FCF_yield_TTM,MA200_slope
2025-02-03,FLNG,25.76,0.3222,0,2.0417,1.3417,0,11.1,12.62,11.08,11.91,10.25,12.16,15.22,9.1,24.32,20.93,22.62,15.98,13.75,14.86,0.0,0.0,0.0,overvalued,-34.29,-100.0,0,-0.13,7.93,3.35,NaN,14.84,7.34,13.63,-0.000093
2025-01-31,FLNG,25.51,0.3222,0,2.0417,1.3417,0,11.1,12.49,11.08,11.91,10.25,12.16,15.22,9.1,24.32,20.93,22.62,15.98,13.75,14.86,0.0,0.0,0.0,overvalued,-34.29,-100.0,0,-0.12,8.00,3.42,NaN,14.84,7.34,13.63,-0.000313
2025-01-30,FLNG,25.87,0.3222,0,2.0417,1.3417,0,11.1,12.67,11.08,11.91,10.25,12.16,15.22,9.1,24.32,20.93,22.62,15.98,13.75,14.86,0.0,0.0,0.0,overvalued,-34.29,-100.0,0,-0.13,7.89,3.31,NaN,14.84,7.34,13.63,-0.000553
2025-01-29,FLNG,25.96,0.3222,0,2.0417,1.3417,0,11.1,12.71,11.08,11.91,10.25,12.16,15.22,9.1,24.32,20.93,22.62,15.98,13.75,14.86,0.0,0.0,0.0,overvalued,-34.29,-100.0,0,-0.13,7.86,3.28,NaN,14.84,7.34,13.63,-0.000920
2025-01-28,FLNG,25.43,0.3222,0,2.0417,1.3417,0,11.1,12.46,11.08,11.91,10.25,12.16,15.22,9.1,24.32,20.93,22.62,15.98,13.75,14.86,0.0,0.0,0.0,overvalued,-34.29,-100.0,0,-0.12,8.03,3.45,NaN,14.84,7.34,13.63,-0.001447
2025-01-27,FLNG,25.11,0.3222,0,2.0417,1.3417,0,11.1,12.30,11.08,11.91,10.25,12.16,15.22,9.1,24.32,20.93,22.62,15.98,13.75,14.86,0.0,0.0,0.0,overvalued,-34.29,-100.0,0,-0.12,8.13,3.55,NaN,14.84,7.34,13.63,-0.002148
2025-01-24,FLNG,25.21,0.3222,0,2.0417,1.3417,0,11.1,12.35,11.08,11.91,10.25,12.16,15.22,9.1,24.32,20.93,22.62,15.98,13.75,14.86,0.0,0.0,0.0,overvalued,-34.29,-100.0,0,-0.12,8.10,3.52,NaN,14.84,7.34,13.63,-0.003008
2025-01-23,FLNG,25.83,0.3222,0,2.0417,1.3417,0,11.1,12.65,11.08,11.91,10.25,12.16,15.22,9.1,24.32,20.93,22.62,15.98,13.75,14.86,0.0,0.0,0.0,overvalued,-34.29,-100.0,0,-0.13,7.90,3.32,NaN,14.84,7.34,13.63,-0.004024
2025-01-22,FLNG,25.69,0.3222,0,2.0417,1.3417,0,11.1,12.58,11.08,11.91,10.25,12.16,15.22,9.1,24.32,20.93,22.62,15.98,13.75,14.86,0.0,0.0,0.0,overvalued,-34.29,-100.0,0,-0.13,7.95,3.37,NaN,14.84,7.34,13.63,-0.005168
2025-01-21,FLNG,25.33,0.3222,0,2.0417,1.3417,0,11.1,12.41,11.08,11.91,10.25,12.16,15.22,9.1,24.32,20.93,22.62,15.98,13.75,14.86,0.0,0.0,0.0,overvalued,-34.29,-100.0,0,-0.12,8.06,3.48,NaN,14.84,7.34,13.63,-0.006412


# Stock screener

In [24]:
ticker_screen_df = ticker_screen_df.sort_values(by=['price_valuation_assessment']
                                                ,ascending=False)

selected_cols = [
    'Ticker'

    ,'BVPS_latest'
    ,'stock_price'
    # Earnings
    # ,'EPS_TTM'

    # PE relative valuation
    ,'PE_TTM'
    ,'PE_TTM_avg'
    ,f"PE_{PE_yr_range-1}yr_avg"
    # ,f"PE_{PE_yr_range-1}yr_volatility_+"
    # ,f"PE_{PE_yr_range-1}yr_volatility_-"
    ,"relative_valuation_TTM_+"
    ,"relative_valuation_TTM_-"
    ,"relative_valuation_TTM_median"
    ,'price_valuation_assessment'

    # Growth
    # ,"EPS_nextQtr_growthRate"
    ,"EPS_nextYr_growthRate"
    ,"PEG_next12months"
    ,"PEG_TTM"

    # Other valuation metrics
    # ,'ERP_TTM'
    # ,'latest_qtr_liquidation_mktcap_ratio_%'
    # ,'PFCF_TTM'
    # ,'FCF_yield_TTM'
    ,'Industry_PE_TTM_avg'
]

ticker_screen_df[selected_cols]

,Ticker,BVPS_latest,stock_price,PE_TTM,PE_TTM_avg,PE_3yr_avg,relative_valuation_TTM_+,relative_valuation_TTM_-,relative_valuation_TTM_median,price_valuation_assessment,EPS_nextYr_growthRate,PEG_next12months,PEG_TTM,Industry_PE_TTM_avg
0,LPG,24.96,23.98,6.07,4.21,8.76,18.21,15.05,16.63,overvalued,-1.01,-1.02,-5.99,13.08
1,STNG,56.62,47.95,3.97,4.64,3.16,64.21,47.8,56.0,fair,-22.95,-0.22,-0.17,13.08
2,GLNG,19.45,41.67,29.21,23.74,35.55,42.35,25.4,33.88,fair,34.57,0.02,0.85,13.08


# Price EPS chart

In [28]:
# Parameters section
alpha_vantage_api_key = API_KEY


ticker_symbols = [
    'STNG'
]



In [29]:
# Daily quote section
for j, symbol in enumerate(ticker_symbols):

    print(j, symbol)
    # STOCK SPLIT FACTOR section
    url = f'https://www.alphavantage.co/query?function=SPLITS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            if len(value) > 0:
                stock_split_record_df = pd.DataFrame(value)
                stock_split_record_df['split_factor'] = pd.to_numeric(stock_split_record_df['split_factor'], errors='coerce') # change split_factor series to numeric data
                stock_split_record_df['effective_date'] = pd.to_datetime(stock_split_record_df['effective_date'])
            else:
                stock_split_record_df = pd.DataFrame()
                stock_split_record_df['split_factor'] = 1
                stock_split_record_df['effective_date'] = datetime.today()


    # Daily quote section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}&outputsize=full'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Time Series (Daily)':


            selected_cols = [
                '4. close'
            ]

            Daily_stock_df = pd.DataFrame(value).transpose()[selected_cols] # tranpose the dataframe and sub select selected cols

            # Rename columns
            Daily_stock_df.rename(
                columns={
                    '4. close': 'stock_price'
                    }
                ,inplace=True
                )
            
            Daily_stock_df["stock_price"] = Daily_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
            Daily_stock_df["stock_price"] = Daily_stock_df["stock_price"].round(2)
            Daily_stock_df.index = pd.to_datetime(Daily_stock_df.index)


    for date_i in Daily_stock_df.index.date:
        for date_j in stock_split_record_df['effective_date'].dt.date:
            if date_i == date_j:

                # stock price to divided the split factor
                Daily_stock_df.loc[Daily_stock_df.index.date < date_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.date == date_j].values[0])




    # Earning section
    # past earnings from alpha vintage API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            annualEPS_df = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


            annualEPS_df['fiscalDateEnding'] = pd.to_datetime(annualEPS_df['fiscalDateEnding']).dt.year


            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    annualEPS_df[f'{col}'] = annualEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue
            
            # clean annualEPS_df
            annualEPS_df = annualEPS_df.sort_values('reportedEPS', ascending=False).drop_duplicates('fiscalDateEnding')
            annualEPS_df = annualEPS_df.sort_values('fiscalDateEnding', ascending=False).reset_index(drop=True)



        if key == 'quarterlyEarnings':

            selected_cols = [
                'reportedDate'
                ,'reportedEPS'
            ]

            qtrEPS_df = pd.DataFrame(value)[selected_cols] # tranpose the dataframe and sub select selected cols
            qtrEPS_df['reportedDate'] = pd.to_datetime(qtrEPS_df['reportedDate'])

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    qtrEPS_df[col] = qtrEPS_df[col].astype(str).apply(lambda x: float(x) if x not in [None, 'None', 'nan', 'NaN'] else float(0))
                else:
                    continue



    # Consolidated section
    stock_consolidate_df = Daily_stock_df
    stock_consolidate_df_date = stock_consolidate_df.index
    for i in stock_consolidate_df_date:
                
        # Filter the DataFrame to include only dates(index) less than or equal to the target date
        filtered_qtrEPS_df = qtrEPS_df[qtrEPS_df['reportedDate'] < i]

        # Select the first four rows from the past_qtrs_EPS
        past_4_qtrs_EPS = filtered_qtrEPS_df.head(4) 
        past_3_qtrs_EPS = filtered_qtrEPS_df.head(3)
        past_1_qtr_EPS = filtered_qtrEPS_df.head(1)

        # Calculate the sum of the numeric values in the selected rows
        EPS_TTM = past_4_qtrs_EPS['reportedEPS'].values.sum()
        EPS_curr_qtr = past_1_qtr_EPS['reportedEPS'].values.sum()

        # assign each index row with the EPS_TTM
        stock_consolidate_df.loc[i, "EPS_TTM"] = EPS_TTM
        stock_consolidate_df.loc[i, "EPS_currentQtr"] = EPS_curr_qtr


    # stock's stats
    stock_consolidate_df["Ticker"] = symbol
    stock_consolidate_df["PE_TTM"] = (stock_consolidate_df["stock_price"] / stock_consolidate_df["EPS_TTM"]).round(2)
    stock_consolidate_df["PE_TTM_avg"] = stock_consolidate_df["PE_TTM"].mean().round(2)
    stock_consolidate_df["PE_TTM_std"] = np.std(stock_consolidate_df["PE_TTM"]).round(2)
    stock_consolidate_df["PE_TTM_volatility_+"] = (stock_consolidate_df["PE_TTM_avg"] + stock_consolidate_df["PE_TTM_std"]).round(2) # 这个是PE的波动范围上限
    stock_consolidate_df["PE_TTM_volatility_-"] = (stock_consolidate_df["PE_TTM_avg"] - stock_consolidate_df["PE_TTM_std"]).round(2) # 这个是PE的波动范围下限

    stock_consolidate_df["relative_valuation_TTM_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * stock_consolidate_df["EPS_TTM"]).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_TTM_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * stock_consolidate_df["EPS_TTM"]).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_TTM_median"] = (np.median([stock_consolidate_df["relative_valuation_TTM_+"][0], stock_consolidate_df["relative_valuation_TTM_-"][0]])).round(2) #这个是根据最新TTM PE估值的价格中位数


0 STNG


In [30]:
import plotly.graph_objects as go


# Create the figure
fig = go.Figure()

# Add stock_price on primary y-axis (left)
fig.add_trace(go.Scatter(
    x=stock_consolidate_df.index,
    y=stock_consolidate_df["stock_price"],
    mode='lines',
    line=dict(color='black'),
    name='Stock Price',
    yaxis="y1"
))

# Add EPS_TTM on secondary y-axis (right)
fig.add_trace(go.Scatter(
    x=stock_consolidate_df.index,
    y=stock_consolidate_df["EPS_TTM"],
    mode='lines',
    fill='tonexty',  # Shadow effect
    line=dict(color='green'),
    name='EPS_TTM',
    yaxis="y2"
))

# Add PE_TTM on secondary y-axis (right)
fig.add_trace(go.Scatter(
    x=stock_consolidate_df.index,
    y=stock_consolidate_df["PE_TTM"],
    mode='lines',
    line=dict(color='blue', dash='dot'),
    name='PE_TTM',
    yaxis="y2"
))

# Update layout to remove grid and configure dual y-axes
fig.update_layout(
    title="Stock Metrics Over Time",
    xaxis=dict(title="Date", showgrid=False),
    yaxis=dict(
        title="Stock Price",
        showgrid=False,
        titlefont=dict(color="black"),
        tickfont=dict(color="black")
    ),
    yaxis2=dict(
        title="EPS_TTM & PE_TTM",
        overlaying="y",  # Overlay with y1
        side="right",
        showgrid=False,
        titlefont=dict(color="blue"),
        tickfont=dict(color="blue")
    ),
    legend_title="Metrics",
    template="plotly_white",
    plot_bgcolor='white'  # Background color
)

# Show the plot
fig.show()

# Financial statement of single stock

In [12]:
stock_financial_dict = {} # the dict will store ticker as key, the string value of 'annual_df' and 'qtr_df' as nested key, actual json as value

ticker_symbols = [
    'SKWD'
    ]
# ticker_symbols = electric_utility_symbols
PE_yr_range = 3 # test for 'X' years PE 


# storage process
for j, symbol in enumerate(ticker_symbols):
    print(j, symbol)
    # COMPANY OVERVIEW
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    
    for key, value in data.items():
        if key  == 'MarketCapitalization':
            stock_mkt_cap = float(value)



    # STOCK SPLIT FACTOR section
    url = f'https://www.alphavantage.co/query?function=SPLITS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            if len(value) > 0:
                stock_split_record_df = pd.DataFrame(value)
                stock_split_record_df['split_factor'] = pd.to_numeric(stock_split_record_df['split_factor'], errors='coerce') # change split_factor series to numeric data
                stock_split_record_df['effective_date'] = pd.to_datetime(stock_split_record_df['effective_date'])
            else:
                stock_split_record_df['split_factor'] = 1
                stock_split_record_df['effective_date'] = datetime.today()



    # Monthly quote section
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Monthly Time Series':
            Monthly_stock_df = pd.DataFrame(value)


    Monthly_stock_df = Monthly_stock_df.transpose()
    Monthly_stock_df.index = pd.to_datetime(Monthly_stock_df.index)


    filter_1 = (Monthly_stock_df.index.year.isin(range((datetime.today().year - PE_yr_range) ,datetime.today().year)))
    filter_2 = (Monthly_stock_df.index.month == 12) # month = 12 to get the year end closing price

    selected_cols = [
        '4. close'
    ]

    Monthly_stock_df = Monthly_stock_df[
        filter_1
        & filter_2
    ][selected_cols]

    # Rename columns
    Monthly_stock_df.rename(
        columns={
            '4. close': 'stock_price'
            }
        ,inplace=True
        )

    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].round(2)

    # modify stock price based on stock split
    for year_i in Monthly_stock_df.index.year:
        for year_j in stock_split_record_df['effective_date'].dt.year:
            if year_i == year_j:

                # stock price to divided the split factor
                Monthly_stock_df.loc[Monthly_stock_df.index.year < year_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.year == year_j].values[0])
    


    # Earning section
    # past earnings from alpha vintage API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            annualEPS_df = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


            annualEPS_df['fiscalDateEnding'] = pd.to_datetime(annualEPS_df['fiscalDateEnding']).dt.year

            annualEPS_df = annualEPS_df[
                annualEPS_df['fiscalDateEnding'].isin(
                    range(
                        (datetime.today().year - PE_yr_range) 
                        ,datetime.today().year
                            )
                            )
                            ]

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    annualEPS_df[f'{col}'] = annualEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue
            # clean annualEPS_df
            annualEPS_df = annualEPS_df.sort_values('reportedEPS', ascending=False).drop_duplicates('fiscalDateEnding')
            annualEPS_df = annualEPS_df.sort_values('fiscalDateEnding', ascending=False).reset_index(drop=True)

            # calculate PE
            annualEPS_df["PE"] = Monthly_stock_df["stock_price"].values / annualEPS_df['reportedEPS'].values




    # INCOME STATEMENT
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()


    for key, value in data.items():
        if key == 'annualReports':
            annual_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_income_df = annual_income_df.sort_values('fiscalDateEnding', ascending=True)

            # annual measurements cols
            annual_income_metric_cols = annual_income_df.columns[1:]

            # Calculate quarter-over-quarter change percentage
            for column in annual_income_metric_cols:  # Exclude the 'fiscalDateEnding' column
                annual_income_df[column] = pd.to_numeric(annual_income_df[column], errors='coerce')
                annual_income_df[f'{column}_YoY'] = annual_income_df[column].pct_change() * 100 
            
            annual_income_YoY_metric_cols = [col for col in annual_income_df.columns if 'YoY' in col]

            # ratios of income statement calculation
            annual_income_df['gross_margin_%'] = (annual_income_df['grossProfit'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['operating_margin_%'] = (annual_income_df['operatingIncome'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['net_margin_%'] = (annual_income_df['netIncome'] / annual_income_df['totalRevenue']) * 100



        if key == 'quarterlyReports':
            qtr_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_income_df = qtr_income_df.sort_values('fiscalDateEnding', ascending=True)

            # qtr measurements cols
            qtr_income_metric_cols = qtr_income_df.columns[1:]
            
            # Calculate quarter-over-quarter change percentage
            for column in qtr_income_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_income_df[column] = pd.to_numeric(qtr_income_df[column], errors='coerce')
                qtr_income_df[f'{column}_QoQ'] = qtr_income_df[column].pct_change() * 100

            qtr_income_QoQ_metric_cols = [col for col in qtr_income_df.columns if 'QoQ' in col]

            # ratios of income statement calculation
            qtr_income_df['gross_margin_%'] = (qtr_income_df['grossProfit'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['operating_margin_%'] = (qtr_income_df['operatingIncome'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['net_margin_%'] = (qtr_income_df['netIncome'] / qtr_income_df['totalRevenue']) * 100

    income_ratio_cols = [
        'gross_margin_%'
        ,'operating_margin_%'
        ,'net_margin_%'
        ,'netIncome'
        ]
    
    
    # BALANCESHEET
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():

        if key == 'annualReports':
            annual_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_balancesheet_df = annual_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in annual_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                annual_balancesheet_df[column] = pd.to_numeric(annual_balancesheet_df[column], errors='coerce')
                annual_balancesheet_df[f'{column}_YoY'] = annual_balancesheet_df[column].pct_change() * 100

            annual_balancesheet_df['current_ratio'] = (annual_balancesheet_df['totalCurrentAssets'] / annual_balancesheet_df['totalCurrentLiabilities'])
            annual_balancesheet_df['working_capital'] = annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['totalCurrentLiabilities']
            annual_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (annual_balancesheet_df['longTermDebt'] / annual_balancesheet_df['working_capital'])
            annual_balancesheet_df['debtEquity_ratio'] = (annual_balancesheet_df['totalLiabilities'] / annual_balancesheet_df['totalShareholderEquity'])
            annual_balancesheet_df['quick_ratio'] = ((annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['inventory']) / annual_balancesheet_df['totalCurrentLiabilities'])

            
        if key == 'quarterlyReports':
            qtr_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_balancesheet_df = qtr_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in qtr_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_balancesheet_df[column] = pd.to_numeric(qtr_balancesheet_df[column], errors='coerce')
                qtr_balancesheet_df[f'{column}_YoY'] = qtr_balancesheet_df[column].pct_change() * 100

            qtr_balancesheet_df['current_ratio'] = (qtr_balancesheet_df['totalCurrentAssets'] / qtr_balancesheet_df['totalCurrentLiabilities'])
            qtr_balancesheet_df['working_capital'] = qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['totalCurrentLiabilities']
            qtr_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (qtr_balancesheet_df['longTermDebt'] / qtr_balancesheet_df['working_capital'])
            qtr_balancesheet_df['debtEquity_ratio'] = (qtr_balancesheet_df['totalLiabilities'] / qtr_balancesheet_df['totalShareholderEquity'])
            qtr_balancesheet_df['quick_ratio'] = ((qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['inventory']) / qtr_balancesheet_df['totalCurrentLiabilities'])


    balancesheet_ratio_cols = [
        'current_ratio'
        ,'working_capital'
        ,'longTermDebt_to_workingCp_ratio'
        ,'debtEquity_ratio'
        ,'quick_ratio'
        ,'totalShareholderEquity'
    ]


    # CASHFLOW STATEMENT
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():

        if key == 'annualReports':
            annual_cashflow_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_cashflow_df = annual_cashflow_df.sort_values('fiscalDateEnding', ascending=True)

            for column in annual_cashflow_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                annual_cashflow_df[column] = pd.to_numeric(annual_cashflow_df[column], errors='coerce')
                annual_cashflow_df[f'{column}_YoY'] = annual_cashflow_df[column].pct_change() * 100

            
        if key == 'quarterlyReports':
            qtr_cashflow_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_cashflow_df = qtr_cashflow_df.sort_values('fiscalDateEnding', ascending=True)

            for column in qtr_cashflow_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_cashflow_df[column] = pd.to_numeric(qtr_cashflow_df[column], errors='coerce')
                qtr_cashflow_df[f'{column}_YoY'] = qtr_cashflow_df[column].pct_change() * 100



    cashflow_ratio_cols = [
        'operatingCashflow'
        ,'capitalExpenditures'
        ,'cashflowFromInvestment'
        ,'cashflowFromFinancing'
        ,'netIncome'
    ]


    ####################
    ### Consolidated ###
    ####################
    stock_ratios_annual_consolidate_df = pd.DataFrame()
    stock_ratios_qtr_consolidate_df = pd.DataFrame()
    stock_PE_annual_consolidate_df = pd.DataFrame()

    stock_ratios_annual_consolidate_df[income_ratio_cols] = annual_income_df[income_ratio_cols]
    stock_ratios_annual_consolidate_df[balancesheet_ratio_cols] = annual_balancesheet_df[balancesheet_ratio_cols]
    stock_ratios_annual_consolidate_df[cashflow_ratio_cols] = annual_cashflow_df[cashflow_ratio_cols]

    stock_ratios_qtr_consolidate_df[income_ratio_cols] = qtr_income_df[income_ratio_cols]
    stock_ratios_qtr_consolidate_df[balancesheet_ratio_cols] = qtr_balancesheet_df[balancesheet_ratio_cols]
    stock_ratios_qtr_consolidate_df[cashflow_ratio_cols] = qtr_cashflow_df[cashflow_ratio_cols]

    # calculating new consolidated metrics 
    # annual df
    stock_ratios_annual_consolidate_df['ROE_%'] = (annual_income_df['netIncome'] / annual_balancesheet_df['totalShareholderEquity']) * 100
    stock_ratios_annual_consolidate_df['liquidation_value'] = ((annual_balancesheet_df['totalAssets'] - annual_balancesheet_df['intangibleAssets']) - annual_balancesheet_df['totalLiabilities'])
    stock_ratios_annual_consolidate_df['liquidation_mktcap_ratio_%'] = round((stock_ratios_annual_consolidate_df['liquidation_value'] / stock_mkt_cap) * 100, 2)
    stock_ratios_annual_consolidate_df['fiscalDateEnding'] = annual_balancesheet_df['fiscalDateEnding']

    stock_ratios_annual_consolidate_df['PE'] = annualEPS_df["PE"]
    stock_ratios_annual_consolidate_df['reportedEPS'] = annualEPS_df["reportedEPS"]

    # YoY %
    stock_ratios_annual_consolidate_df['QoQ_gross_margin_%_%'] = stock_ratios_annual_consolidate_df["gross_margin_%"].pct_change() * 100
    stock_ratios_annual_consolidate_df['QoQ_operating_margin_%_%'] = stock_ratios_annual_consolidate_df["operating_margin_%"].pct_change() * 100
    stock_ratios_annual_consolidate_df['QoQ_net_margin_%_%'] = stock_ratios_annual_consolidate_df["net_margin_%"].pct_change() * 100
    stock_ratios_annual_consolidate_df['YoY_reportedEPS_%'] = stock_ratios_annual_consolidate_df["reportedEPS"].pct_change() * 100 # yearly EPS 的增速



    # qtr df
    stock_ratios_qtr_consolidate_df['ROE_%'] = (qtr_income_df['netIncome'] / qtr_balancesheet_df['totalShareholderEquity']) * 100
    stock_ratios_qtr_consolidate_df['liquidation_value'] = ((qtr_balancesheet_df['totalAssets'] - qtr_balancesheet_df['intangibleAssets']) - qtr_balancesheet_df['totalLiabilities'])
    stock_ratios_qtr_consolidate_df['liquidation_mktcap_ratio_%'] = round((stock_ratios_qtr_consolidate_df['liquidation_value'] / stock_mkt_cap) * 100, 2)
    stock_ratios_qtr_consolidate_df['fiscalDateEnding'] = qtr_balancesheet_df['fiscalDateEnding']
    
    # QoQ %
    stock_ratios_qtr_consolidate_df['QoQ_gross_margin_%_%'] = stock_ratios_qtr_consolidate_df["gross_margin_%"].pct_change() * 100
    stock_ratios_qtr_consolidate_df['QoQ_operating_margin_%_%'] = stock_ratios_qtr_consolidate_df["operating_margin_%"].pct_change() * 100
    stock_ratios_qtr_consolidate_df['QoQ_net_margin_%_%'] = stock_ratios_qtr_consolidate_df["net_margin_%"].pct_change() * 100




    # annual PE df
    if stock_PE_annual_consolidate_df.empty:
        stock_PE_annual_consolidate_df['fiscalDateEnding'] = annualEPS_df['fiscalDateEnding']

    stock_PE_annual_consolidate_df[f'{symbol}_PE'] = annualEPS_df["PE"].round(2)



    # reformat the columns order
    stock_ratios_annual_consolidate_df = stock_ratios_annual_consolidate_df[['fiscalDateEnding'] + [col for col in stock_ratios_annual_consolidate_df.columns if col != 'fiscalDateEnding']]
    stock_ratios_qtr_consolidate_df = stock_ratios_qtr_consolidate_df[['fiscalDateEnding'] + [col for col in stock_ratios_qtr_consolidate_df.columns if col != 'fiscalDateEnding']]




    # store the stock, dataframe value pair to the dictionary
    # transfer the pandas df to json
    stock_ratios_annual_consolidate_json = stock_ratios_annual_consolidate_df.to_dict()
    stock_ratios_qtr_consolidate_json = stock_ratios_qtr_consolidate_df.to_dict()

    stock_financial_dict[symbol] = {
    'annual': stock_ratios_annual_consolidate_json
    ,'qtr': stock_ratios_qtr_consolidate_json
    }

0 SKWD


ValueError: operands could not be broadcast together with shapes (2,) (3,) 

In [263]:
qtr_balancesheet_df

,fiscalDateEnding,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,accumulatedDepreciationAmortizationPPE,intangibleAssets,intangibleAssetsExcludingGoodwill,goodwill,investments,longTermInvestments,shortTermInvestments,otherCurrentAssets,otherNonCurrentAssets,totalLiabilities,totalCurrentLiabilities,currentAccountsPayable,deferredRevenue,currentDebt,shortTermDebt,totalNonCurrentLiabilities,capitalLeaseObligations,longTermDebt,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding,totalAssets_YoY,totalCurrentAssets_YoY,cashAndCashEquivalentsAtCarryingValue_YoY,cashAndShortTermInvestments_YoY,inventory_YoY,currentNetReceivables_YoY,totalNonCurrentAssets_YoY,propertyPlantEquipment_YoY,accumulatedDepreciationAmortizationPPE_YoY,intangibleAssets_YoY,intangibleAssetsExcludingGoodwill_YoY,goodwill_YoY,investments_YoY,longTermInvestments_YoY,shortTermInvestments_YoY,otherCurrentAssets_YoY,otherNonCurrentAssets_YoY,totalLiabilities_YoY,totalCurrentLiabilities_YoY,currentAccountsPayable_YoY,deferredRevenue_YoY,currentDebt_YoY,shortTermDebt_YoY,totalNonCurrentLiabilities_YoY,capitalLeaseObligations_YoY,longTermDebt_YoY,currentLongTermDebt_YoY,longTermDebtNoncurrent_YoY,shortLongTermDebtTotal_YoY,otherCurrentLiabilities_YoY,otherNonCurrentLiabilities_YoY,totalShareholderEquity_YoY,treasuryStock_YoY,retainedEarnings_YoY,commonStock_YoY,commonStockSharesOutstanding_YoY,current_ratio,working_capital,longTermDebt_to_workingCp_ratio,debtEquity_ratio,quick_ratio
61,2009-07-05,3493422000,1178924000,28768000,28768000,642505000,NaN,2314498000,1440530000,NaN,688937000,125315000,563622000,NaN,NaN,NaN,235109000,163849000,3036242000,1001939000,NaN,NaN,NaN,245434000,2034303000,NaN,1504475000,16495000,NaN,1504475000,460304000,503638000,417756000,-4.008120e+09,3991467000,299192000,60710000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.176642,176985000,8.500579,7.267979,0.535381
60,2009-09-30,3773107000,1462637000,119253000,119253000,559318000,NaN,2310470000,1412818000,1.935216e+09,692508000,125345000,567163000,NaN,NaN,NaN,216457000,180368000,3135590000,1108329000,2.852310e+08,NaN,NaN,227389000,2027261000,NaN,1503435000,15632000,NaN,1746456000,243021000,481105000,597070000,3.989117e+09,4087572000,299192000,60709,8.006047,24.065419,314.533509,314.533509,-12.947292,NaN,-0.174033,-1.923736,NaN,0.518335,0.023940,0.628258,NaN,NaN,NaN,-7.933342,10.081844,3.272071,10.618411,NaN,NaN,NaN,-7.352282,-0.346163,NaN,-0.069127,-5.231888,NaN,16.084082,-47.204239,-4.474047,42.923142,-199.525887,2.407761,0.000000,-99.900002,1.319678,354308000,4.243300,5.251629,0.815028
59,2009-12-31,3675031000,1385434000,253605000,253605000,519712000,NaN,2289597000,1404767000,1.838101e+09,697100000,125520000,571580000,NaN,NaN,NaN,201727000,183377000,2914692000,910628000,2.879350e+08,NaN,NaN,24066000,2004064000,NaN,1517977000,15247000,NaN,1542043000,130200000,501334000,720459000,3.979629e+09,4148353000,299192000,242653870,-2.599343,-5.278343,112.661317,112.661317,-7.081124,NaN,-0.903409,-0.569854,-5.018303,0.663097,0.139615,0.778788,NaN,NaN,NaN,-6.805047,1.668256,-7.044862,-17.837754,0.948004,NaN,NaN,-89.416375,-1.144253,NaN,0.967252,-2.462897,NaN,-11.704446,-46.424383,4.204695,20.665751,-0.237847,1.486971,0.000000,399599.995058,1.521405,474806000,3.197047,4.045604,0.950687
58,2010-03-31,3698687000,1413032000,303786000,303786000,481854000,NaN,2285655000,1394678000,1.842224e+09,703039000,125327000,577712000,NaN,NaN,NaN,216147000,180619000,2909622000,902295000,2.942230e+08,NaN,NaN,42162000,2007327000,NaN,1502183000,13786000,NaN,1558131000,466288000,500504000,752217000,4.014954e+09,4224841000,299192000,60709,0.643695,1

In [264]:
qtr_income_df

,fiscalDateEnding,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome,grossProfit_QoQ,totalRevenue_QoQ,costOfRevenue_QoQ,costofGoodsAndServicesSold_QoQ,operatingIncome_QoQ,sellingGeneralAndAdministrative_QoQ,researchAndDevelopment_QoQ,operatingExpenses_QoQ,investmentIncomeNet_QoQ,netInterestIncome_QoQ,interestIncome_QoQ,interestExpense_QoQ,nonInterestIncome_QoQ,otherNonOperatingIncome_QoQ,depreciation_QoQ,depreciationAndAmortization_QoQ,incomeBeforeTax_QoQ,incomeTaxExpense_QoQ,interestAndDebtExpense_QoQ,netIncomeFromContinuingOperations_QoQ,comprehensiveIncomeNetOfTax_QoQ,ebit_QoQ,ebitda_QoQ,netIncome_QoQ,gross_margin_%,operating_margin_%,net_margin_%
65,2008-06-30,382511000,1105437000,722926000,722926000,94113000,266612000,NaN,266612000,NaN,NaN,NaN,23610000,NaN,NaN,NaN,56791000,70503000,29036000,NaN,NaN,NaN,137685000,194476000,41467000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.602696,8.513647,3.751186
64,2008-09-30,501229000,1489609000,988380000,988380000,219951000,272401000,NaN,272401000,NaN,NaN,NaN,24915000,NaN,NaN,NaN,65674000,195036000,70498000,NaN,NaN,NaN,237705000,303379000,124538000,31.036493,34.752953,36.719388,36.719388,133.709477,2.171320,NaN,2.171320,NaN,NaN,NaN,5.527319,NaN,NaN,NaN,15.641563,176.635037,142.795151,NaN,NaN,NaN,72.644079,55.998169,200.330383,33.648360,14.765687,8.360449
63,2008-12-31,497526000,1377380000,879854000,879854000,153416000,284057000,28100000.0,284057000,NaN,NaN,NaN,26682000,NaN,NaN,NaN,58729000,128451000,46296000,NaN,NaN,NaN,180523000,239252000,82155000,-0.738784,-7.534125,-10.980190,-10.980190,-30.249919,4.278986,NaN,4.278986,NaN,NaN,NaN,7.092113,NaN,NaN,NaN,-10.574961,-34.139851,-34.330052,NaN,NaN,NaN,-24.055868,-21.137587,-34.032183,36.121187,11.138248,5.964585
62,2009-04-05,440228000,1236031000,795803000,795803000,152934000,274456000,NaN,274456000,NaN,NaN,NaN,24770000,NaN,NaN,NaN,46877000,129038000,53144000,NaN,NaN,NaN,166646000,213523000,75894000,-11.516584,-10.262164,-9.552835,-9.552835,-0.314178,-3.379955,0.000000,-3.379955,NaN,NaN,NaN,-7.165880,NaN,NaN,NaN,-20.180831,0.456984,14.791775,NaN,NaN,NaN,-7.687109,-10.753933,-7.620960,35.616259,12.372991,6.140137
61,2009-07-05,453290000,1171183000,1016603000,717893000,-899927000,298710000,NaN,319010000,NaN,NaN,NaN,22734000,NaN,NaN,NaN,47181000,93942000,22644000,NaN,71298000.0,112701000.0,-899927000,154580000,71298000,2.967099,-5.246470,27.745560,-9.790111,-688.441419,8.837118,0.000000,16.233567,NaN,NaN,NaN,-8.219621,NaN,NaN,NaN,0.648506,-27.198190,-57.391239,NaN,NaN,NaN,-640.023163,-27.604989,-6.055815,38.703601,-76.839145,6.087691
60,2009-09-30,589098000,1484118000,1196486000,895020000,-916862000,301466000,NaN,313466000,NaN,-23227000.0,NaN,23227000,NaN,NaN,NaN,44816000,257322000,95299000,23227000.0,162023000.0,214946000.0,280549000,287632000,162023000,29.960511,26.719565,17.694518,24.673176,1.881819,0.922634,0.000000,-1.737877,NaN,NaN,NaN,2.168558,NaN,NaN,NaN,-5.012611,173.915820,320.857622,NaN,127.247609,90.722354,-131.174640,86.073231,127.247609,39.693475,-61.778241,10.917124
59,2009-12-31,570521000,1407336000,1214392000,836815000,570521000,334040000,28100000.0,262384000,NaN,NaN,NaN,21711000,NaN,NaN,NaN,43537000,190829000,64050000,NaN,570521000.0,172063000.0,570521000,280202000,126779000,-3.153465,-5.173578,1.496549,-6.503207,-162.225395,10.805199,0.000000,-16.295866,NaN,0.000000,NaN,-6.526887,NaN,NaN,NaN,-2.853891,-25.840387,-32.790480,0.000000,252.123464,-19.950592,103.358772,-2.583162,-21.752467,40.539075,40.539075,9.008439
58,2010-03-31,593980000,1407843000,1201839000,813863000,593980000,

In [266]:
pd.DataFrame(stock_financial_dict['SKWD']['qtr']).tail(20)

,fiscalDateEnding,gross_margin_%,operating_margin_%,net_margin_%,netIncome,current_ratio,working_capital,longTermDebt_to_workingCp_ratio,debtEquity_ratio,quick_ratio,totalShareholderEquity,operatingCashflow,capitalExpenditures,cashflowFromInvestment,cashflowFromFinancing,ROE_%,liquidation_value,liquidation_mktcap_ratio_%,QoQ_gross_margin_%_%,QoQ_operating_margin_%_%,QoQ_net_margin_%_%
19,2019-12-31,44.093660,13.838525,10.018108,271137000.0,1.053917,1.083090e+08,39.093732,3.677162,0.648076,1.739222e+09,246093000.0,99217000.0,-1.106690e+08,495625000.0,11.912625,-1.582127e+09,-5.24,-0.230381,-35.902146,-34.268643
18,2020-03-31,42.537416,18.789418,13.308533,268901000.0,0.986848,-3.804600e+07,-111.502313,4.296774,0.699171,1.671231e+09,367954000.0,86567000.0,-9.913300e+07,-199767000.0,16.223790,-1.626955e+09,-5.39,-3.529406,35.776156,32.844770
17,2020-06-30,46.420579,22.454547,15.749806,447283000.0,1.310692,6.881260e+08,7.091229,4.080156,0.859467,1.753605e+09,481219000.0,106267000.0,-1.314520e+08,-315823000.0,15.334183,-1.537245e+09,-5.09,9.128817,19.506351,18.343667
16,2020-09-30,48.653477,27.543113,20.149435,291387000.0,1.439326,9.789030e+08,4.986705,3.478572,1.009164,2.065854e+09,604391000.0,149575000.0,-1.900260e+08,-479263000.0,21.651240,-1.216478e+09,-4.03,4.810149,22.661624,27.934499
15,2020-12-31,44.025793,18.538662,13.334300,395799000.0,1.574158,1.086160e+09,4.169353,3.085441,1.064466,2.234352e+09,609640000.0,114471000.0,-1.370050e+08,-491862000.0,13.041231,-1.045546e+09,-3.46,-9.511518,-32.692204,-33.822956
14,2021-03-31,46.122475,24.317397,17.403320,301230000.0,1.611615,1.085392e+09,4.098182,3.002349,1.098944,2.255994e+09,408012000.0,113136000.0,-5.631150e+08,-549829000.0,17.544329,-1.008727e+09,-3.34,4.762395,31.171264,30.515431
13,2021-06-30,46.810430,23.099323,15.236676,444927000.0,1.398348,6.321790e+08,6.483401,2.918246,0.730156,2.265049e+09,386065000.0,119843000.0,-1.396190e+08,3865000.0,13.299050,-1.401988e+09,-4.64,1.491582,-5.009065,-12.449599
12,2021-09-30,44.917027,24.327568,18.829868,335556000.0,1.438212,8.354490e+08,4.905112,2.729924,0.899768,2.528294e+09,679167000.0,148427000.0,-1.383089e+09,356690000.0,17.597914,-1.121553e+09,-3.71,-4.044830,5.317235,23.582519
11,2021-12-31,43.671501,19.797990,14.468073,533478000.0,0.900892,-2.471070e+08,-16.549394,2.776339,0.504427,2.757229e+09,656467000.0,141063000.0,-1.639660e+08,-463454000.0,12.170045,-1.913533e+09,-6.34,-2.772949,-18.619118,-23.164234
10,2022-03-31,46.426535,26.875636,19.885935,315556000.0,0.957828,-1.081990e+08,-37.807789,2.651128,0.555782,2.915549e+09,457306000.0,99897000.0,-1.870190e+08,-291951000.0,18.297686,-1.734265e+09,-5.74,6.308540,35.749319,37.447018


In [267]:
# searching & merge process
YoY_EPS_screener = pd.DataFrame()

for symbol in stock_financial_dict.keys():
    annaul_df = pd.DataFrame(
        stock_financial_dict[symbol]['annual']
    )

    annaul_df = annaul_df[['fiscalDateEnding', 'YoY_reportedEPS_%', 'reportedEPS']].rename(
        columns={
            'YoY_reportedEPS_%': f'{symbol}_YoY_reportedEPS_%'
            ,'reportedEPS': f'{symbol}_reportedEPS'
            }
    )
    annaul_df['fiscalDateEnding'] = pd.to_datetime(annaul_df['fiscalDateEnding']).dt.year


    if YoY_EPS_screener.empty:
        YoY_EPS_screener = annaul_df
    else:
        YoY_EPS_screener = pd.merge(
            YoY_EPS_screener
            ,annaul_df
            ,on='fiscalDateEnding'
            ,how='outer'
        )


In [268]:
YoY_EPS_screener

,fiscalDateEnding,HSY_YoY_reportedEPS_%,HSY_reportedEPS
14,2009,NaN,NaN
13,2010,NaN,NaN
12,2011,NaN,NaN
11,2012,NaN,NaN
10,2013,NaN,NaN
9,2014,NaN,4.12
8,2015,7.038835,4.41
7,2016,7.936508,4.76
6,2017,12.605042,5.36
5,2018,8.022388,5.79


In [269]:
stock_PE_annual_consolidate_df

,fiscalDateEnding,HSY_PE
0,2024,25.35
1,2023,19.44
2,2022,27.18
3,2021,26.95
4,2020,24.22
5,2019,25.39
6,2018,20.00
7,2017,23.85
8,2016,23.45
9,2015,21.67


# SP500 sectors

In [203]:
# Function to get the list of S&P 500 companies and their sectors
def get_sp500_companies():
    # Fetch the S&P 500 company symbols and sectors from a reliable source (e.g., Wikipedia)
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    tables = pd.read_html(url)
    
    # Extract the relevant table containing the company symbols and sectors
    sp500_df = tables[0]
    
    # Return the DataFrame containing S&P 500 companies and sectors
    return sp500_df[['Symbol', 'GICS Sector', 'GICS Sub-Industry']]


# Function to create a dictionary of sectors and sub-sectors
def create_sector_subsector_dict(df):
    sector_subsector_dict = {}
    for index, row in df.iterrows():
        sector = row['GICS Sector']
        subsector = row['GICS Sub-Industry']
        if sector not in sector_subsector_dict:
            sector_subsector_dict[sector] = [subsector]
        else:
            sector_subsector_dict[sector].append(subsector)
    return sector_subsector_dict

# Function to filter the S&P 500 companies by sector
def company_sector_list(df, sector):
    return df[df['GICS Sector'] == sector]['Symbol'].tolist()

def company_sub_sector_list(df, sub_sector):
    return df[df['GICS Sub-Industry'] == sub_sector]['Symbol'].tolist()


# Get the list of S&P 500 companies and their sectors
sp500_df  = get_sp500_companies()

sp500_companies_sectors = sp500_df ['GICS Sector'].value_counts().index
sp500_companies_sub_sectors = sp500_df ['GICS Sub-Industry'].value_counts().index

sector_subsector_dict = create_sector_subsector_dict(sp500_df)


# Function to create a DataFrame from the sector_subsector_dict
def create_sector_dataframe():
    # Create a list to store dictionacompany_sector_listries representing each row of data
    data = []
    
    # Filter the DataFrame to get stocks in the specified sector
    for sector in sp500_companies_sectors:
        sector_stocks_list = company_sector_list(sp500_df, sector)

        # Iterate over the stocks in the sector and create a dictionary for each
        for i, ticker in enumerate(sector_stocks_list, start=1):
            # Create a dictionary for the current stock in the sector
            row_data = {'Sector': sector, 'Ticker': ticker}
            # Append the dictionary to the list
            data.append(row_data)
    
    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    return df


sector_subsector_dict = create_sector_subsector_dict(sp500_df)


sector_ticker_df = create_sector_dataframe()
# sector_ticker_list = sector_ticker_df[sector_ticker_df['Sector'] == 'Consumer Staples']['Ticker'].tolist()

In [204]:
# Pivot the DataFrame
sp500_pivot_sector_ticker_df = sector_ticker_df.groupby('Sector')['Ticker'].apply(list).reset_index()

# Transpose to get sectors as columns
sp500_pivot_sector_ticker_df = sp500_pivot_sector_ticker_df.set_index('Sector').T

sp500_pivot_sector_ticker_df

Sector,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
Ticker,"[GOOGL, GOOG, T, CHTR, CMCSA, EA, FOXA, FOX, I...","[ABNB, AMZN, APTV, AZO, BBY, BKNG, BWA, CZR, K...","[MO, ADM, BF.B, BG, CPB, CHD, CLX, KO, CL, CAG...","[APA, BKR, CVX, COP, CTRA, DVN, FANG, EOG, EQT...","[AFL, ALL, AXP, AIG, AMP, AON, APO, ACGL, AJG,...","[ABT, ABBV, A, ALGN, AMGN, BAX, BDX, TECH, BII...","[MMM, AOS, ALLE, AME, ADP, AXON, BA, BR, BLDR,...","[ACN, ADBE, AMD, AKAM, APH, ADI, ANSS, AAPL, A...","[APD, ALB, AMCR, AVY, BALL, CE, CF, CTVA, DOW,...","[ARE, AMT, AVB, BXP, CPT, CBRE, CSGP, CCI, DLR...","[AES, LNT, AEE, AEP, AWK, ATO, CNP, CMS, ED, C..."


In [205]:
sp500_pivot_sector_ticker_df['Consumer Discretionary'].iloc[0]

['ABNB',
 'AMZN',
 'APTV',
 'AZO',
 'BBY',
 'BKNG',
 'BWA',
 'CZR',
 'KMX',
 'CCL',
 'CMG',
 'DRI',
 'DECK',
 'DPZ',
 'DHI',
 'EBAY',
 'EXPE',
 'F',
 'GRMN',
 'GM',
 'GPC',
 'HAS',
 'HLT',
 'HD',
 'LVS',
 'LEN',
 'LKQ',
 'LOW',
 'LULU',
 'MAR',
 'MCD',
 'MGM',
 'MHK',
 'NKE',
 'NCLH',
 'NVR',
 'ORLY',
 'POOL',
 'PHM',
 'RL',
 'ROST',
 'RCL',
 'SBUX',
 'TPR',
 'TSLA',
 'TJX',
 'TSCO',
 'ULTA',
 'WYNN',
 'YUM']

In [207]:
len(sp500_pivot_sector_ticker_df['Consumer Discretionary'].iloc[0])

50

# Russell 1000 sectors

In [208]:
# Function to get the list of S&P 500 companies and their sectors
def get_rusell1000_companies():
    # Fetch the S&P 500 company symbols and sectors from a reliable source (e.g., Wikipedia)
    url = 'https://en.wikipedia.org/wiki/Russell_1000_Index'
    tables = pd.read_html(url)
    
    # Extract the relevant table containing the company symbols and sectors
    rs1000_df = tables[3]
    
    # Return the DataFrame containing S&P 500 companies and sectors
    return rs1000_df[['Company', 'Symbol', 'GICS Sector', 'GICS Sub-Industry']]



# Function to create a dictionary of sectors and sub-sectors
def create_sector_subsector_dict(df):
    sector_subsector_dict = {}
    for index, row in df.iterrows():
        sector = row['GICS Sector']
        subsector = row['GICS Sub-Industry']
        if sector not in sector_subsector_dict:
            sector_subsector_dict[sector] = [subsector]
        else:
            sector_subsector_dict[sector].append(subsector)
    return sector_subsector_dict

# Function to filter the S&P 500 companies by sector
def company_sector_list(df, sector):
    return df[df['GICS Sector'] == sector]['Symbol'].tolist()

def company_sub_sector_list(df, sub_sector):
    return df[df['GICS Sub-Industry'] == sub_sector]['Symbol'].tolist()


# Get the list of S&P 500 companies and their sectors
rs1000_df  = get_rusell1000_companies()

rs1000_companies_sectors = rs1000_df ['GICS Sector'].value_counts().index
rs1000_companies_sub_sectors = rs1000_df ['GICS Sub-Industry'].value_counts().index

sector_subsector_dict = create_sector_subsector_dict(rs1000_df)


# Function to create a DataFrame from the sector_subsector_dict
def create_sector_dataframe():
    # Create a list to store dictionacompany_sector_listries representing each row of data
    data = []
    
    # Filter the DataFrame to get stocks in the specified sector
    for sector in rs1000_companies_sectors:
        sector_stocks_list = company_sector_list(rs1000_df, sector)

        # Iterate over the stocks in the sector and create a dictionary for each
        for i, ticker in enumerate(sector_stocks_list, start=1):
            # Create a dictionary for the current stock in the sector
            row_data = {'Sector': sector, 'Ticker': ticker}
            # Append the dictionary to the list
            data.append(row_data)
    
    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    return df


sector_subsector_dict = create_sector_subsector_dict(rs1000_df)


sector_ticker_df = create_sector_dataframe()

In [209]:
# Pivot the DataFrame
rs1000_pivot_sector_ticker_df = sector_ticker_df.groupby('Sector')['Ticker'].apply(list).reset_index()

# Transpose to get sectors as columns
rs1000_pivot_sector_ticker_df = rs1000_pivot_sector_ticker_df.set_index('Sector').T

rs1000_pivot_sector_ticker_df

Sector,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
Ticker,"[GOOGL, GOOG, T, CHTR, CMCSA, EA, FOXA, FOX, F...","[ADT, AAP, ABNB, AMZN, AS, APTV, ARMK, AN, AZO...","[ACI, MO, ADM, BRBR, BJ, SAM, BF.A, BF.B, BG, ...","[AM, AR, APA, BKR, LNG, CVX, CHRD, CIVI, COP, ...","[AMG, AFL, AGNC, ALL, ALLY, AXP, AFG, AIG, AMP...","[TXG, ABT, ABBV, ACHC, A, ALGN, ALNY, AMED, AM...","[MMM, AOS, AAON, AYI, WMS, ACM, AGCO, AL, ALK,...","[ACN, ADBE, AMD, AFRM, AKAM, ALGM, DOX, AMKR, ...","[APD, ALB, AA, AMCR, ATR, ASH, ATI, AVY, AXTA,...","[ADC, ARE, AMH, AMT, COLD, AVB, BRX, BXP, CPT,...","[AES, LNT, AEE, AEP, AWK, ATO, AGR, BEPC, CNP,..."


In [210]:
set(rs1000_pivot_sector_ticker_df['Consumer Discretionary'].iloc[0]) -  set(sp500_pivot_sector_ticker_df['Consumer Discretionary'].iloc[0])

{'AAP',
 'ADT',
 'AN',
 'ARMK',
 'AS',
 'BBWI',
 'BC',
 'BFAM',
 'BIRK',
 'BLD',
 'BROS',
 'BURL',
 'BYD',
 'CAVA',
 'CHDN',
 'CHH',
 'COLM',
 'CPNG',
 'CPRI',
 'CRI',
 'CROX',
 'CVNA',
 'DASH',
 'DDS',
 'DKNG',
 'DKS',
 'DUOL',
 'ETSY',
 'FIVE',
 'FND',
 'GAP',
 'GME',
 'GNTX',
 'H',
 'HOG',
 'HRB',
 'JWN',
 'KSS',
 'LAD',
 'LCID',
 'LEA',
 'LEG',
 'LEN.B',
 'LNW',
 'LOPE',
 'M',
 'MAT',
 'MTN',
 'MUSA',
 'NWL',
 'OLLI',
 'PAG',
 'PENN',
 'PII',
 'PLNT',
 'PVH',
 'QS',
 'RH',
 'RIVN',
 'SCI',
 'SKX',
 'SN',
 'THO',
 'TNL',
 'TOL',
 'TPX',
 'TXRH',
 'UA',
 'UAA',
 'VAC',
 'VFC',
 'VSTS',
 'W',
 'WEN',
 'WH',
 'WHR',
 'WING',
 'WSM',
 'YETI'}

In [211]:
len(rs1000_pivot_sector_ticker_df['Consumer Discretionary'].iloc[0])

129

# Stock PE by years by tickers

In [281]:
stock_financial_dict = {} # the dict will store ticker as key, the string value of 'annual_df' and 'qtr_df' as nested key, actual json as value
stock_PE_annual_consolidate_df = pd.DataFrame()

ticker_symbols = [
'MNST'
,'KO'
,'KDP'
,'PEP'
 ]
# ticker_symbols = electric_utility_symbols
PE_yr_range = 10 # test for 'X' years PE 

In [282]:

# storage process
for j, symbol in enumerate(ticker_symbols):
    print(j, symbol)
    # COMPANY OVERVIEW
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    
    for key, value in data.items():
        if key  == 'MarketCapitalization':
            stock_mkt_cap = float(value)



    # STOCK SPLIT FACTOR section
    url = f'https://www.alphavantage.co/query?function=SPLITS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            if len(value) > 0:
                stock_split_record_df = pd.DataFrame(value)
                stock_split_record_df['split_factor'] = pd.to_numeric(stock_split_record_df['split_factor'], errors='coerce') # change split_factor series to numeric data
                stock_split_record_df['effective_date'] = pd.to_datetime(stock_split_record_df['effective_date'])
            else:
                stock_split_record_df['split_factor'] = 1
                stock_split_record_df['effective_date'] = datetime.today()



    # Monthly quote section
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Monthly Time Series':
            Monthly_stock_df = pd.DataFrame(value)


    Monthly_stock_df = Monthly_stock_df.transpose()
    Monthly_stock_df.index = pd.to_datetime(Monthly_stock_df.index)


    filter_1 = (Monthly_stock_df.index.year.isin(range((datetime.today().year - PE_yr_range) ,datetime.today().year)))
    filter_2 = (Monthly_stock_df.index.month == 12) # month = 12 to get the year end closing price

    selected_cols = [
        '4. close'
    ]

    Monthly_stock_df = Monthly_stock_df[
        filter_1
        & filter_2
    ][selected_cols]

    # Rename columns
    Monthly_stock_df.rename(
        columns={
            '4. close': 'stock_price'
            }
        ,inplace=True
        )

    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].round(2)

    # modify stock price based on stock split
    for year_i in Monthly_stock_df.index.year:
        for year_j in stock_split_record_df['effective_date'].dt.year:
            if year_i == year_j:

                # stock price to divided the split factor
                Monthly_stock_df.loc[Monthly_stock_df.index.year < year_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.year == year_j].values[0])
    


    # Earning section
    # past earnings from alpha vintage API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            annualEPS_df = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


            annualEPS_df['fiscalDateEnding'] = pd.to_datetime(annualEPS_df['fiscalDateEnding']).dt.year

            annualEPS_df = annualEPS_df[
                annualEPS_df['fiscalDateEnding'].isin(
                    range(
                        (datetime.today().year - PE_yr_range) 
                        ,datetime.today().year
                            )
                            )
                            ]

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    annualEPS_df[f'{col}'] = annualEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue
            # clean annualEPS_df
            annualEPS_df = annualEPS_df.sort_values('reportedEPS', ascending=False).drop_duplicates('fiscalDateEnding')
            annualEPS_df = annualEPS_df.sort_values('fiscalDateEnding', ascending=False).reset_index(drop=True)

            # calculate PE
            annualEPS_df["PE"] = Monthly_stock_df["stock_price"].values / annualEPS_df['reportedEPS'].values


    ####################
    ### Consolidated ###
    ####################

    # annual PE df
    if stock_PE_annual_consolidate_df.empty:
        stock_PE_annual_consolidate_df['fiscalDateEnding'] = annualEPS_df['fiscalDateEnding']

    stock_PE_annual_consolidate_df[f'{symbol}_PE'] = annualEPS_df["PE"].round(2)


stock_PE_annual_consolidate_df['industry_avg_PE'] = stock_PE_annual_consolidate_df.iloc[:, 1:].sum(axis=1) / (stock_PE_annual_consolidate_df.shape[1] -1)  # Sum by row


0 MNST
1 KO
2 KDP
3 PEP


In [283]:
stock_PE_annual_consolidate_df

,fiscalDateEnding,MNST_PE,KO_PE,KDP_PE,PEP_PE,industry_avg_PE
0,2024,43.44,26.72,23.97,24.53,29.6650
1,2023,36.46,21.91,18.61,22.29,24.8175
2,2022,22.76,25.65,21.23,26.61,24.0625
3,2021,18.68,25.41,23.04,27.79,23.7300
4,2020,17.25,28.12,22.86,26.87,23.7750
5,2019,15.35,26.23,23.73,24.76,22.5175
6,2018,13.67,22.66,8.90,19.55,16.1950
7,2017,22.77,24.02,22.11,23.20,23.0250
8,2016,17.05,21.71,20.61,29.98,22.3375
9,2015,25.08,21.48,23.18,21.91,22.9125


## Appendix

In [131]:
# Consumer Staples

ticker_symbols = [
    'MO',
    'ADM',
    # 'BF.B', # DATA ISSUE
    'BG',
    'CPB',
    'CHD',
    'CLX',
    'KO',
    'CL',
    'CAG',
    'STZ', # !!
    'COST',
    'DG',
    'DLTR', # !!
    'EL',
    'GIS',
    'HSY',
    'HRL',
    'K',
    'KVUE',
    'KDP',
    'KMB',
    'KHC',
    'KR', # DATA ISSUE
    'LW',
    'MKC', # DATA ISSUE
    'TAP',
    'MDLZ',
    'MNST',
    'PEP',
    'PM',
    'PG',
    'SJM',
    'SYY',
    'TGT', # !!
    'TSN', # !!
    'WBA', # DATA ISSUE
    'WMT'
]

In [132]:
# Energy

ticker_symbols = [
'APA',
 'BKR', # !! HIGH PE, LOW PROFITABILITY, HIGH FORECAST,  BUT CANNOT BEAT FORECAST, 总结与TRGP类似, 基本面更稳妥, 盈利预期高, 有待考证
 'CVX',
 'COP',
 'CTRA',
 'DVN',
 'FANG',
 'EOG',
 'EQT',
 'XOM',
 'HAL', # !!!! 与SLB类似
 'HES', # !  HIGH PE, UNDERVALUE, 市场炒作高预期, 2024表现强劲, 已经被超热了
 'KMI',
 'MRO', # PROFITABILITY, STEADY GROWTH FORECAST, AND CAN BEAT FORECAST, 总结基本面良好，与HAL类型相似, 市场炒作热度比HAL更高, 值得研究
 'TRGP', # !！ HGIH PE
 'MPC',
 'OXY',
 'OKE', # !!
 'PSX',
 'SLB', # !!! A BIT HIGH PE, GOOD ROE, MED, HIGH ROE, RELATIVE MORE DEBT THAN EQUITY, LOW PROFITABILITY, HIGH FORECAST, BUT CANNOT BEAT FORECAST, 总结基本面一般, 但是盈利预期非常高, 有待考证
 'VLO',
 'WMB'
 ]

In [135]:
electric_utility_symbols = [
'CEPU'
,'OTTR'
,'PAM'
,'SO' ##
,'DUK'
,'NRG'
,'IDA'
,'DTE' #
,'CEG' ##
,'SRE'
,'ETR'
,'PEG'
,'MGEE'
,'LNT'
,'BKH'
]